In [113]:
from mxnet import autograd,nd

num_inputs = 2
num_size = 10000
true_w = [2,-3.2]
true_b = 3.14
features = nd.random.normal(scale=1,shape=(num_size,num_inputs))


In [114]:
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1]+true_b
labels += nd.random.normal(scale=1,shape=labels.shape)

In [115]:
from mxnet.gluon import data as gdata

batch_size = 100
dataset = gdata.ArrayDataset(features,labels)
data_iter = gdata.DataLoader(dataset,batch_size,shuffle=True)


In [116]:
for x,y in data_iter:
    print(x,y)
    break


[[ 1.0079473  -0.92134404]
 [-0.1713012  -2.0279496 ]
 [ 0.17844214  1.2301452 ]
 [ 1.9136347  -0.2552128 ]
 [-1.3627725   0.29322797]
 [-0.7284469  -0.28493327]
 [ 0.455459   -1.804643  ]
 [ 0.21885672 -0.6000325 ]
 [-0.33976445 -1.0621741 ]
 [ 0.8249318  -0.8975398 ]
 [ 0.15928647 -1.2259878 ]
 [ 0.5304128   0.79565233]
 [-0.05923939  0.4199652 ]
 [ 0.6312917   0.2981324 ]
 [ 1.6292155  -0.8225457 ]
 [ 1.0946896  -0.75263286]
 [-1.3622835  -0.15089722]
 [-0.30975538  0.49526563]
 [-0.70850074 -0.46241572]
 [-1.3878338  -1.5773487 ]
 [-1.0841727   0.2738722 ]
 [ 0.64483684  0.64754814]
 [ 0.22348242  0.8039566 ]
 [ 1.2408583  -0.2700648 ]
 [-0.2594816  -1.1977301 ]
 [-0.9683042   0.47630695]
 [-1.4225953   0.34209248]
 [ 0.45120686  0.16028827]
 [-1.2357953   0.0454513 ]
 [-0.18503524 -0.38905144]
 [ 0.43760237  0.42225602]
 [ 0.8279446   0.7741935 ]
 [-0.60103893 -0.63199747]
 [-0.75594324  0.35959494]
 [-1.0656114   0.08963902]
 [-0.5513665  -0.6802282 ]
 [ 0.44665855  0.16457188]


In [117]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

In [118]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

In [119]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

In [120]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.01})

In [121]:
num_epochs = 10
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features),labels)
    print('epoch %d,loss: %f ' % (epoch,l.mean().asnumpy()))

epoch 1,loss: 2.089010 
epoch 2,loss: 0.704697 
epoch 3,loss: 0.523008 
epoch 4,loss: 0.499194 
epoch 5,loss: 0.495832 
epoch 6,loss: 0.495516 
epoch 7,loss: 0.495421 
epoch 8,loss: 0.495414 
epoch 9,loss: 0.495422 
epoch 10,loss: 0.495417 


In [122]:
dense = net[0]
dense.weight.data()


[[ 1.9933279 -3.1966467]]
<NDArray 1x2 @cpu(0)>

In [123]:
dense.bias.data()


[3.1334164]
<NDArray 1 @cpu(0)>